In [1]:
%config IPCompleter.greedy=True
import re
import json
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk
from pymystem3 import Mystem
from sklearn.feature_extraction.text import CountVectorizer
import requests
from time import time
import csv

## Connect

In [2]:
es = Elasticsearch(
    "https://localhost:9200",
    ca_certs="./http_ca.crt",
    basic_auth=("elastic", "RikVUty_*6bSkVMxGifL")
)


## Create index

### Create empty index

In [ ]:
es.indices.create(index='wikir')

### Create configuration

We can describe documents structure and specify the way it must be processed using `mappings`.

Under `mapping.properties` we define fields of our index. For each field we have to specify its type. Common types are:

- **text**: general purpose string type
- **numeric** family
- **date**
- **boolean**
- **keyword**: allows only full-match search
- **object**/**nested**: stores complex JSON objects. **nested** support related field search, while **object** merges objects array field by field
- **rank_feature**: keeps a number which is used when computing relevance

Note that any type of field support arrays of this type out-of-the-box. The reason is that search engine has to deal with sequential data anyway, so array support could be implemented for free via using existing methods.

In [4]:
settings_1 = {
    'mappings': {
        'properties': {
            'content': {
                'type': 'text'
            },
            'doc_id': {
                'type': 'numeric'
            },
        }
    }
}

#### Let us use analysis!

What is analyzer?

Search engine actually indexes a sequence of tokens. Analyzer is a module which takes raw content and returns resulting tokens depending on the our goal in every single case. These are the tokens which will be the keys in inverted index.

Analyzer's pipeline looks like

 \*content to be indexed\* ->  
 ***0 or more*** *character filters* ->  
 ***exactly 1*** *tokenizer* ->  
 ***0 or more*** *token filters* ->  
 \*actually indexing\*

1. *Character filter* edits characters, lol. The only common one I know is an HTML stripper.
2. *Tokenizer* splits text to tokens, surprisingly.
    - **standard**: some tricky algorithm, you should really be careful using with Russian
    - **letter**: performing split each time it sees non-letter symbol
    - **whitespace**: your choice if tokenization has been done
    - **pattern**: splits by separator defined as Java regex
    - **other**
3. *Token filter* changes token stream, usually only changes or remove tokens. Therefore, it sometimes adds tokens, for example, in case of query expansion.
    - **lowercase**
    - **shingle**
    - **stop**: removes stopwords, there is built-in support for many languages (not quite reliable, though)
    - **stemmer** and **snowball**: for russian they are both snowball, but I'm not really sure if they are indentical
    - **hunspell**: provides morphological analysis, but it is very limited due to dictionary-based algorithm
    
Also, we have some ready-to-use analyzers:  
- **standard**: default analyzer, standard tokenizer + lowercase
- **simple**: letter tokenizer + lowercase
- **whitespace**: plain whitespace tokenizer
- **keyword**: fairly does nothing
- **pattern**: pattern tokenizer with optional lowercase
- **language**: language-specific pipeline, but definitely not a silver bullet

#### Set it up!

Now what if we want to build a custom analyzer. We can do that using `setting.analysis` parameter.

Suppose we've already done with morphological analysis, so we don't want to use any character filter and the best tokenizer is plain whitespace with limiting possible token length to 20. But, we still need lowercasing. Let's do that!

In [5]:
settings_3 = {
    'mappings': {
        'properties': {
            'content': {
                'type': 'text'
            },
            'doc_id': {
                'type': 'numeric'
            },
        }
    },
    'settings': {
        'analysis': {
            'analyzer': {
                'white_lover': {
                    'tokenizer': 'white_20',
                    'filter': [
                        'lowercase'
                    ]
                }
            },
            'tokenizer': {
                'white_20': {
                    'type': 'whitespace',
                    'max_token_length': 5
                }
            }
        }
    }
}

Now let's consider more complex setting. Here's what we want this time:

1. Split text to tokens which consist of 2 or more word characters.
2. Lowercase everything.
3. Remove some set of stopwords.
4. Apply stemming with english snowball.

Well, this will require a bit of work.

In [8]:
settings_4 = {
    'mappings': {
        'properties': {
            'content': {
                'type': 'text'
            },
            'doc_id': {
                'type': 'numeric'
            },
        }
    },
    'settings': {
        'analysis': {
            'tokenizer': {
                'word_longer_2': {
                    'type': 'pattern',
                    'pattern': '\w{2,}',
                    'group': 0
                }
            },
            'filter': {
                'au_stop': {
                    'type': 'stop',
                    'stopwords': stopwords
                },
            },
        }
    }
}

Now we want to use created custom analyzers to process certain document fields. Note if we assign an analyzer to a field, it will be applied to document content at index-time and also to query text at query-time (if query is asked to the same field). 

Also sometimes we want to handle the same field with different analyzers. We have to used `fields` property for that purpose, setting one analyzer per-subfield.

Let's analyze `content` field with *white_lover* analyzer and `content` field with default analyzer, creating two subfields `white` and `complex` with *white_lover* and *russian_complex* analyzers respectively. We will be able to refer these fields in queries as `content.white` and `content.complex`.

In [112]:
settings_final = {
    'mappings': {
        'properties': {
            'content': {
                'type': 'text',
                'fields': {
                    'white': {
                        'type': 'text',
                        'analyzer': 'white_lover'
                    },
                }
            },
            'doc_id': {
                'type': 'integer'
            }
        }
    },
    'settings': {
        'analysis': {
            'tokenizer': {
                'word_longer_2': {
                    'type': 'pattern',
                    'pattern': '\w{2,}',
                    'group': 0
                },
                'white_20': {
                    'type': 'whitespace',
                    'max_token_length': 5
                }
            },
            'filter': {
                'english_snow': {
                    'type': 'snowball',
                    'language': 'english'
                },
            },
            'analyzer': {
                'white_lover': {
                    'tokenizer': 'white_20',
                },
                'stemmer': {
                    'tokenizer': 'word_longer_2',
                    'filter': [
                        'english_snow'
                    ]
                },
            },
        
        }
    }
}

### Create index with proper configuration

We are ready to use index setting. Let's define a function which allows us to easily update index settings.

In [113]:
def recreate_index(with_stemming=False):
    es.indices.delete(index='wikir')
    if with_stemming:
        settings_final['mappings']['properties']['content']['fields']['stemmed'] = {
            'type': 'text',
            'analyzer': 'stemmer'
        }
    es.indices.create(index='wikir', body=settings_final)

In [143]:
start = time()
recreate_index(with_stemming=False)
print(f"Time spent without stemming {time() - start:0.2f} seconds")

start = time()
recreate_index(with_stemming=True)
print(f"Time spent with stemming {time() - start:0.2f} seconds")


/var/folders/k6/3zkmms5n2j3_c0wt5q78hm040000gn/T/ipykernel_46833/294914630.py:8: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index='wikir', body=settings_final)


Time spent without stemming 1.88 seconds
Time spent with stemming 0.15 seconds


#### Check custom analyzers

After we set up analysis we want to check if it works in proper way. We can do this using Elastic `analyze` API:

In [115]:
def check_analyzer(analyzer, text):
    body = analyzer
    body['text'] = text
    
    tokens = es.indices.analyze(index='wikir', body=body)['tokens']
    tokens = [token_info['token'] for token_info in tokens]
    return tokens

In [116]:
text = 'a barcode is a machine readable optical label that contains information about the item'

We have to describe analyzer in any way and send it to `check_analyzer`. Let's check **standard** analyzer first.

In [117]:
analyzer = {
    'analyzer': 'standard'
}

check_analyzer(analyzer, text)

/var/folders/k6/3zkmms5n2j3_c0wt5q78hm040000gn/T/ipykernel_46833/1309655745.py:5: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  tokens = es.indices.analyze(index='wikir', body=body)['tokens']


['a',
 'barcode',
 'is',
 'a',
 'machine',
 'readable',
 'optical',
 'label',
 'that',
 'contains',
 'information',
 'about',
 'the',
 'item']

And now check **whitespace**.

In [118]:
analyzer = {
    'analyzer': 'whitespace'
}

check_analyzer(analyzer, text)

/var/folders/k6/3zkmms5n2j3_c0wt5q78hm040000gn/T/ipykernel_46833/1309655745.py:5: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  tokens = es.indices.analyze(index='wikir', body=body)['tokens']


['a',
 'barcode',
 'is',
 'a',
 'machine',
 'readable',
 'optical',
 'label',
 'that',
 'contains',
 'information',
 'about',
 'the',
 'item']

Note that if we apply `analyze` query to some index, we can use analyzers, tokenizers and filters described inside it:

In [119]:
analyzer = {
    'analyzer': 'white_lover'
}

check_analyzer(analyzer, text)

/var/folders/k6/3zkmms5n2j3_c0wt5q78hm040000gn/T/ipykernel_46833/1309655745.py:5: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  tokens = es.indices.analyze(index='wikir', body=body)['tokens']


['a',
 'barco',
 'de',
 'is',
 'a',
 'machi',
 'ne',
 'reada',
 'ble',
 'optic',
 'al',
 'label',
 'that',
 'conta',
 'ins',
 'infor',
 'matio',
 'n',
 'about',
 'the',
 'item']

We got results, although **white_lover** is not built-in.

Try with **english**.

In [120]:
analyzer = {
    'analyzer': 'stemmer'
}

check_analyzer(analyzer, text)

/var/folders/k6/3zkmms5n2j3_c0wt5q78hm040000gn/T/ipykernel_46833/1309655745.py:5: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  tokens = es.indices.analyze(index='wikir', body=body)['tokens']


['barcod',
 'is',
 'machin',
 'readabl',
 'optic',
 'label',
 'that',
 'contain',
 'inform',
 'about',
 'the',
 'item']

Well done, it is all good now!

## Index documents

At this point we want to add documents to the index. The easiest way to do this is using `parallel_bulk` API. First of all, we have to create a function, which builds an Elastic *action*. *Action* is actually just an index entry, which consist of several meta-fields. We will be focused on 3 of them. `_id` field is literally unique document identificator. `_index` field shows which index the document belongs to. And `_source` field contains document data itself as a JSON object. Let's code it.

In [121]:
def create_es_action(index, doc_id, document):
    return {
        '_index': index,
        '_id': doc_id,
        '_source': document
    }

Now we have to get some iterable of actions. The most appropriate solution in many cases is creating a generator function. I have my data JSON-represented, so generator will be quite simple:

In [122]:
def es_actions_generator():
    with open('./wikIR1k/documents.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader, None)
        for row in reader:
            doc_id = int(row[0])
            doc = {
                'content': row[1],
                'doc_id': doc_id,
            }
            yield create_es_action('wikir', doc_id, doc)

And finally we run indexing.

In [140]:
# without stemming
start = time()
for ok, result in parallel_bulk(es, es_actions_generator(), queue_size=4, thread_count=4, chunk_size=1000):
    if not ok:
        print(result)
print(f"Time spent without stemming {time() - start:0.2f} seconds")

Time spent without stemming 51.71 seconds


In [148]:
# with stemming
start = time()
for ok, result in parallel_bulk(es, es_actions_generator(), queue_size=4, thread_count=4, chunk_size=1000):
    if not ok:
        print(result)
print(f"Time spent with stemming {time() - start:0.2f} seconds")


Time spent with stemming 56.60 seconds


## Search

Here we are, ready to perform search!

We will use `search` API, which takes query as a JSON object and returns a responce as a JSON object too. Let's define a pair of useful functions for visualization of results.

- Run test queries, get top20 results for each query, estimate query execution time
- Save triples `<queryID, docID, score>` for two variants

In [152]:
def run_test_queries(filename):
    with open('./wikIR1k/test/queries.csv', 'r') as csvfile:
        with open(f"{filename}.csv", 'w') as file:
            writer = csv.writer(file)
            writer.writerow(['queryID', 'docID', 'score'])
            reader = csv.reader(csvfile)
            next(reader, None)
            for row in reader:
                query_id = int(row[0])
                query = {
                    'query': {
                        'bool': {
                            'should': {
                                'match_phrase': {
                                    'content': row[1]
                                }
                            }
                        }
                    }
                }
                res = es.search(index='wikir', body=query, size=20)['hits']
                for hit in res['hits']:
                        data = [query_id, hit['_id'], hit['_score']]
                        writer.writerow(data)


In [153]:
start = time()
run_test_queries("variant1")
print(f"Time spent without stemming {time() - start:0.2f} seconds")

/var/folders/k6/3zkmms5n2j3_c0wt5q78hm040000gn/T/ipykernel_46833/1766794919.py:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index='wikir', body=query, size=20)['hits']


Time spent without stemming 1.14 seconds


In [151]:
start = time()
run_test_queries("variant2")
print(f"Time spent with stemming {time() - start:0.2f} seconds")

/var/folders/k6/3zkmms5n2j3_c0wt5q78hm040000gn/T/ipykernel_46833/2765061889.py:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index='wikir', body=query, size=20)['hits']


Time spent with stemming 1.19 seconds


## Evaluation

Consider search engine $E$ gives documents sequence $(d_1, .., d_n)$ by query $q$. Then define $E(q) = (d_1, .., d_n)$.

We use binary relevance model where relevance function $rel(q, d)$ returns 0 or 1.

Let $k$ be positive number. Define number of relevant documents in the first k $s_k(q) = \sum\limits_1^k rel(q, E(q)_k)$. Define total number of relevant documents by query $q$ as $R(q) = \sum\limits_{d \in collection} rel(q, d)$

Also define *precision at level k*  $p_{@k}(q) = \frac{s_k(q)}{k}$ and *recall at level k* $r_{@k}(q) = \frac{s_k(q)}{R(q)}$.

If we let $k = R(q)$ then we get $p_{@k}(q) = r_{@k}(q)$. This is called *R-precision*.

Since now we didn't pay attention to ranking. We want to give higher weights to relevant documents at top and lower weights to documents at bottom. We will aggregate $p_{@i}$ by $i$ from $1$ to $k$ and it leads us to *Mean Average Precision*.

$MAP_{@k} = \frac{1}{k} \sum\limits_1^k rel(q, E(q)_k)$

In fact, we usually evaluate a system with a bunch of queries, so the formulas will be:

$p_{@k}(q) = \frac{1}{|Q|} \sum\limits_{q \in Q} \frac{s_k(q)}{k}$

$r_{@k}(q) = \frac{1}{|Q|} \sum\limits_{q \in Q} \frac{s_k(q)}{R(q)}$

$MAP_{@k} = \frac{1}{|Q|} \sum\limits_{q \in Q} \frac{1}{k} \sum\limits_1^k rel(q, E(q)_k)$

In [154]:
import ir_measures
from ir_measures import *

In [180]:
def generate_trec(filename):
    with open(f"{filename}.csv", 'r') as csvfile:
        with open(f"{filename}.qrels.csv", 'w') as qrels_file:
            with open(f"{filename}.run.csv", 'w') as run_file:
                reader = csv.reader(csvfile)
                qrels_writer = csv.writer(qrels_file, delimiter=' ')
                run_writer = csv.writer(run_file, delimiter=' ')

                next(reader, None)

                i = 0
                queryId = 0
                j = 0
                for row in reader:
                    qrels_writer.writerow(
                        [row[0], i, row[1], int(row[0] != row[1])])

                    if row[0] != queryId:
                        queryId = row[0]
                        j = 0

                    run_writer.writerow([row[0], 0, row[1], j, row[2], 'BM25'])
                    i += 1
                    j += 1


In [181]:
generate_trec("variant1")
generate_trec("variant2")


In [198]:
qrels = ir_measures.read_trec_qrels('./variant1.qrels.csv')
run = ir_measures.read_trec_run('./variant1.run.csv')
ir_measures.calc_aggregate([P@10, P@20, MAP], qrels, run)


{AP: 0.9319861300671777, P@10: 0.8353535353535356, P@20: 0.7712121212121219}

In [199]:
qrels = ir_measures.read_trec_qrels('./variant2.qrels.csv')
run = ir_measures.read_trec_run('./variant2.run.csv')
ir_measures.calc_aggregate([P@10, P@20, MAP], qrels, run)


{AP: 0.9308992979521602, P@10: 0.8434343434343435, P@20: 0.7838383838383847}

In [200]:
qrels = ir_measures.read_trec_qrels('./wikIR1k/test/qrels')
run = ir_measures.read_trec_run('./wikIR1k/test/BM25.res')
ir_measures.calc_aggregate([P@10, P@20, MAP], qrels, run)


{AP: 0.11196168401599797, P@10: 0.1319999999999999, P@20: 0.09499999999999999}